In [ ]:
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

NUMBER_OF_NEURONS = 10
LEARNING_RATE = 1.0e-3
NUMBER_OF_EPOCHS = 10000

In [ ]:
# Reset restrictions on the number of output rows
pd.set_option('display.max_rows', 10)
 
# Reset restrictions on the number of columns
pd.set_option('display.max_columns', 7)
 
# Reset restrictions on the number of characters in the record
pd.set_option('display.max_colwidth', 100)

In [ ]:
wine = sklearn.datasets.load_wine()
pd.DataFrame(wine.data, columns=[wine.feature_names])

In [ ]:
pd.DataFrame(wine.target, columns =['wine_sort'])

In [ ]:
x_whole = wine.data[:,:2]
y_whole = wine.target
x_train, x_test, y_train, y_test = train_test_split(
    wine.data[:,:2],
    wine.target,
    test_size = 0.3,
    shuffle=True)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self, n_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(2, n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, n_neurons)
        self.ac2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)
        return x
    
    
    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
wine_net = WineNet(NUMBER_OF_NEURONS)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(wine_net.parameters(), lr = LEARNING_RATE)

In [ ]:
batch_size = 10

for epoch in range(NUMBER_OF_EPOCHS):
    order = np.random.permutation(len(x_train))
    for start_index in range(0, len(x_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = x_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch)
        
        loss_val = loss(preds, y_batch)
        loss_val.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(x_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)

n_classes = 3
plot_colors = ['green', 'orange', 'black']
plot_step = 0.02

x_min, x_max = x_train[:, 0].min()-1, x_train[:, 0].max()+1
y_min, y_max = x_train[:, 1].min()-1, x_train[:, 1].max()+1

xx, yy = torch.meshgrid(torch.arange(x_min, x_max, plot_step),
                        torch.arange(y_min, y_max, plot_step))

preds = wine_net.inference(
    torch.cat([xx.reshape(-1, 1), yy.reshape(-1, 1)], dim=1))

preds_class = preds.data.numpy().argmax(axis=1)
preds_class = preds_class.reshape(xx.shape)
plt.contourf(xx, yy, preds_class, cmap='Accent')

for i, color in zip(range(n_classes), plot_colors):
    indexes = np.where(y_train == i)
    plt.scatter(x_train[indexes, 0],
                x_train[indexes, 1],
                c=color,
                label=wine.target_names[i],
                cmap='Accent')
    plt.xlabel(wine.feature_names[0])
    plt.ylabel(wine.feature_names[1])
    plt.legend()